# Faults sources within an area source
Given an area source and a set of faults we find the faults whose trace intersects the polygon of the area source and computes the percentage of the fault trace inside the polygon.

In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner ; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
%matplotlib notebook 

import os
import re
import sys
import numpy
from prettytable import PrettyTable
import cPickle as pickle

from mbt.tools.geo import get_idx_points_inside_polygon, get_line_inside_polygon

In [ ]:
try:
    plot_figures
except: 
    plot_figures = False
    
try:
    print_log
except: 
    print_log = False

In [ ]:
# Load source
#model_key = metys.g_prj.models.current 
#data_path = os.path.join(metys.g_prj.folder, '%s_area_sources.pkl' % model_key)
#fin = open(data_path,'rb') 
#sources = pickle.load(fin)
#fin.close()

# Check input variables
#if area_source_ids_list is None:
#    area_source_ids_list = ['10']
# Check that the number of sources to process is equal to one
if len(area_source_ids_list) > 1:
    assert 0 == 1
# Get source data
src = sources[area_source_ids_list[0]]
print 'Processing area source with ID:', area_source_ids_list[0]

## Selected faults

In [ ]:
from prettytable import PrettyTable

idx_faults_in_polygon = []
frac_faults_in_polygon = {}

# Setting table header
p = PrettyTable(["ID","Name","Fraction","Slip rate", "RI"])
p.align["ID"] = 'l'
p.align["Name"] = 'l'
p.align["Fraction"] = 'r'
p.align["Slip rate"] = 'r'
p.align["RI"] = 'r'
p.padding_width = 1
p.float_format = '.2'

assert len(area_source_ids_list) == 1
area_source = sources[area_source_ids_list[0]]
area_source.ids_faults_inside = {}

# Processing faults
for key in sources:

    fault = sources[key]
    if fault.source_type == 'SimpleFaultSource':
        plons = []
        plats = []
        pdeps = []
        
        # Create lists with the coordinates of the points defining the trace
        for pnt in fault.trace.points:
            plons.append(pnt.longitude)
            plats.append(pnt.latitude)
            pdeps.append(pnt.depth)

        # Find points on the trace within the polygon
        iii = get_idx_points_inside_polygon(plons, 
                                            plats, 
                                            area_source.polygon.lons, 
                                            area_source.polygon.lats, 
                                            numpy.arange(0, len(plons)+1))

        frac = get_line_inside_polygon(plons, 
                                       plats, 
                                       area_source.polygon.lons, 
                                       area_source.polygon.lats)

        if frac > 0.0 and fault.__dict__.has_key('slip_rate'):
            
            idx_faults_in_polygon.append(fault.source_id)
            frac_faults_in_polygon[fault.source_id] = frac
            recurrence = fault.recurrence if hasattr(fault, 'recurrence') else None
            slip_rate = fault.slip_rate if hasattr(fault, 'slip_rate') else None
            p.add_row([fault.source_id, fault.name, frac, slip_rate, recurrence])
        
            if hasattr(area_source, 'ids_faults_inside'):
                area_source.ids_faults_inside[fault.source_id] = frac
            else:
                area_source.ids_faults_inside = {fault.source_id: frac} 
                
        if frac > 0.5:
            fault.id_intersecting_area = area_source_ids_list[0]
            
print 'Faults intersecting the area source %s\n' % (area_source_ids_list[0])        
print p

sources[area_source_ids_list[0]] = area_source